# Some basic operatioins by using Pytorch

In [2]:
import torch
import torch as nn
import numpy as np

## Create torch tensors

In [12]:
### Basic creator
x = torch.tensor([1, 2, 3])
print(x.shape)

### Random creator
print(torch.rand(3, 5)) # 生成0-1的随机数
print(torch.randn(3, 5)) # 生成标准正态分布的随机数

### From numpy

torch.Size([3])
tensor([[0.7925, 0.2678, 0.8116, 0.3457, 0.3045],
        [0.7637, 0.0065, 0.0652, 0.5085, 0.8011],
        [0.7650, 0.4977, 0.5035, 0.5900, 0.6967]])
tensor([[ 2.5541, -0.1705, -0.7015,  0.5510, -0.0155],
        [ 0.4679, -0.2098,  0.6918,  2.0037, -0.1771],
        [-1.1810,  1.9495, -0.7814, -0.5482, -0.7604]])


## Different data type conversion

In [10]:
pred = torch.FloatTensor(np.array([1,2,3]))
print(pred.dtype)
print(pred)
pred = pred.to(torch.int64)
print(pred.dtype)
print(pred)


torch.float32
tensor([1., 2., 3.])
torch.int64
tensor([1, 2, 3])


In [5]:
labels = None
labels = torch.tensor([1, 2, 3])
print(labels.dtype)
if labels is None: # use 'is' instead of ==
    print("Labels is None")
else:
    print("Labels is not None")

torch.int64
Labels is not None


In [13]:
## Loop the Tensor
data_a = torch.randn(3, 5, 5, 5)
print(data_a)


tensor([[[[ 1.2346e+00,  3.7939e-01, -1.9367e+00,  1.3094e+00,  4.6663e-01],
          [ 3.4819e-01, -5.6895e-01, -1.5108e+00,  7.0780e-01,  9.0777e-01],
          [ 4.6561e-01,  2.1563e+00, -2.3255e+00, -6.6472e-01,  5.5147e-01],
          [ 9.4951e-01, -3.3740e-01, -6.2161e-02,  4.8511e-01, -1.4107e-01],
          [ 6.3537e-01,  1.5464e+00,  2.0419e+00,  2.4523e-01,  2.2240e+00]],

         [[-5.4798e-01,  1.6411e-01, -2.4302e-01, -2.1069e+00, -1.1748e+00],
          [ 5.8614e-01, -1.0792e+00,  8.6184e-02,  2.6984e-02,  3.7680e-01],
          [-4.2325e-02,  5.9603e-01, -5.9892e-01,  6.5933e-01,  9.7178e-01],
          [-1.4920e+00, -2.1150e-01, -2.5007e-01, -2.6350e-01, -7.3031e-01],
          [-8.8017e-01, -1.0805e+00, -8.5367e-01,  2.3745e+00,  1.7426e+00]],

         [[ 3.5111e-01, -1.0492e+00, -6.6652e-01,  1.4015e+00,  2.0608e+00],
          [-1.2575e+00, -4.0794e-01,  2.2955e-02, -1.0286e+00,  6.4215e-01],
          [-8.6350e-01, -2.7992e-01, -2.8991e-01,  9.1334e-01,  3.0354e-

In [19]:
### Create meshgrid
# Ray helpers
def get_rays(H, W, K, c2w):
    i, j = torch.meshgrid(torch.linspace(0, W-1, W), torch.linspace(0, H-1, H))  # pytorch's meshgrid has indexing='ij'
    i = i.t()
    j = j.t()
    dirs = torch.stack([(i-K[0][2])/K[0][0], -(j-K[1][2])/K[1][1], -torch.ones_like(i)], -1)

    # Rotate ray directions from camera frame to the world frame
    rays_d = torch.sum(dirs[..., np.newaxis, :] * c2w[:3,:3], -1)  # dot product, equals to: [c2w.dot(dir) for dir in dirs]
    # Translate camera frame's origin to the world frame. It is the origin of all rays.
    rays_o = c2w[:3,-1].expand(rays_d.shape)
    return rays_o, rays_d

H, W, focal = 400, 400, 555.55
K = np.array([
            [focal, 0, 0.5*W],
            [0, focal, 0.5*H],
            [0, 0, 1]
        ])
c2w = np.random.rand(3, 4)
get_rays(H, W, K, c2w)

torch.Size([400, 400, 3])
torch.Size([400, 3])


## Slice operations for Tensor

In [12]:
data_a = torch.rand((2, 21, 48, 48))
length = 10
data_a_fw = data_a[:, :length-1, ...]
print(data_a_fw.shape)

data_a_bw = data_a[:, length:, ...].flip(1)
print(data_a_bw.shape)

t = 19
t = t // 3 * 3
print(t)

n = 2
n_clip = n * t // 3
print(n_clip)

hr_flow = list(range(19))
hr_flow_bw = hr_flow[0:t:3]
print(hr_flow_bw)

torch.Size([2, 9, 48, 48])
torch.Size([2, 11, 48, 48])
18
12
[1, 4, 7, 10, 13, 16]


## Dimensions operation

In [27]:
### squeeze和unsqueeze的用法 https://pytorch.org/docs/stable/generated/torch.unsqueeze.html
x = torch.tensor([1, 2, 3, 4])
print(torch.unsqueeze(x, 0))
print(torch.unsqueeze(x, 1))

### Expand函数不复制张量元素
x = torch.tensor([1, 2, 3])
x = x.expand(2, 3)
print(x, x.shape)

### Repeat函数复制张量元素
y = torch.tensor([1, 2, 3])
y = y.repeat(2, 3)
print(y, y.shape)

### 使用Repeat可以为张量增加维度
y = torch.tensor([1, 2, 3])
print(y.shape)
y = y.unsqueeze(0).repeat(5, 1, 1)
print(y.shape)
print(y)

tensor([[1, 2, 3, 4]])
tensor([[1],
        [2],
        [3],
        [4]])
tensor([[1, 2, 3],
        [1, 2, 3]]) torch.Size([2, 3])
tensor([[1, 2, 3, 1, 2, 3, 1, 2, 3],
        [1, 2, 3, 1, 2, 3, 1, 2, 3]]) torch.Size([2, 9])
torch.Size([3])
torch.Size([5, 1, 3])
tensor([[[1, 2, 3]],

        [[1, 2, 3]],

        [[1, 2, 3]],

        [[1, 2, 3]],

        [[1, 2, 3]]])


## Misc math operations

## Conca, Stack operations

In [4]:
### 连接堆叠操作
data_a = torch.randn(2, 3, 5, 96, 96)
data_b = torch.randn(2, 3, 5, 96, 96)
data_ab = torch.cat([data_a[:,i] for i in range(data_a.size(1))], dim=0)
print(data_ab.shape)

data_a = [torch.rand(5), torch.rand(5)]
data_b = torch.stack(data_a, 1)
print(data_b.shape)

data_a = [torch.rand((2,3)), torch.rand((2,3)), torch.rand((2,3))]
data_b = torch.stack(data_a, dim=0)
print("Stack result: \n", data_b, data_b.shape)

torch.Size([6, 5, 96, 96])
torch.Size([5, 2])
Stack result: 
 tensor([[[0.1713, 0.7808, 0.6824],
         [0.3596, 0.4430, 0.6680]],

        [[0.3666, 0.3668, 0.5351],
         [0.9481, 0.5435, 0.8422]],

        [[0.5633, 0.0573, 0.2535],
         [0.7198, 0.6823, 0.8020]]]) torch.Size([3, 2, 3])


In [33]:
a=torch.tensor([[[1,2,3],[4,5,6]],
                [[7,8,9],[10,11,12]]])
print("a的shape：",a.shape)
#在第0维上进行split
b=torch.split(a, 1)
print("b:", b, len(b))
#在第1维上进行split
c=torch.split(a,[1,1],1)
print("c:",c)

a的shape： torch.Size([2, 2, 3])
b: (tensor([[[1, 2, 3],
         [4, 5, 6]]]), tensor([[[ 7,  8,  9],
         [10, 11, 12]]])) 2
c: (tensor([[[1, 2, 3]],

        [[7, 8, 9]]]), tensor([[[ 4,  5,  6]],

        [[10, 11, 12]]]))


In [16]:
gt_with_border = torch.rand((2, 10,3,128, 128))
border_size = int(1.5*3.0)
print(border_size)

scale = 4
n, t, c, gt_h, gt_w = gt_with_border.size()
lr_h = (gt_h - 2*border_size)//scale
lr_w = (gt_w - 2*border_size)//scale
print(lr_h, lr_w)

gt_data = gt_with_border[
                ...,
                border_size: border_size + scale*lr_h,
                border_size: border_size + scale*lr_w
            ]
print(gt_data.shape)

4
30 30
torch.Size([2, 10, 3, 120, 120])
